<a href="https://colab.research.google.com/github/redinbluesky/nlp-with-transformers/blob/main/04-다중_언어_개체명_인식.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  텍스트 분류 목차
* [Chapter 0 개요](#chapter0)

## Chapter 0 개요 <a class="anchor" id="chapter0"></a>
1. NLP를 적용할 문서가 다른 언어이거나, 다국어로 되어있는 경우 문제가 발생할 수 있다.
    - 러시아어, 중국어, 독일어와 같은 대표적인 언어는 허깅페이스에서 적절한 사전 훈련된 언어 모델을 찾아 미세튜닝 가능하다.
    - 그리스어, 스와힐리어 등의 언어는 사전 훈련된 모델을 찾기 어려울 수 있다.

2. 다중 언어 트랜스포머 모델이 등장하면서, 하나의 모델로 여러 언어를 처리할 수 있게 되었다.
    - 대표적인 다중 언어 트랜스포머 모델: mBERT, XLM-RoBERTa
    - 다중 언어 트랜스포머 모델은 여러 언어로 된 대규모 말뭉치를 사용해 사전 훈련되었다.
    - 다중 언어 트랜스포머 모델은 단일 언어 모델에 비해 성능이 약간 떨어질 수 있지만, 다양한 언어를 처리할 수 있다는 장점이 있다.

3. 다중 언어 트랜스포머는 많은 언어로 된 대규모 말뭉치에서 사전 훈련해서 제로샷 교차 언어 전이가 가능하다.
    - 제로샷 교차 언어 전이: 모델이 훈련되지 않은 언어로 된 문서에 대해 예측할 수 있는 능력
    - 예를 들어, 영어로 된 데이터셋으로 미세튜닝한 모델이 러시아어로 된 문서에 대해서도 예측할 수 있다.

4. 이번 장에서는 XML-RoBERTa 모델을 사용해 다중 언어 개체명 인식(NER) 작업을 수행한다.
    - 개체명 인식(NER): 문서에서 사람, 장소, 조직 등의 개체를 식별하는 작업
    - 다중 언어 NER 작업에서는 여러 언어로 된 문서에서 개체를 식별해야 한다.
    - 예를 들어 회사 문서에서 중요한 정보를 추출하거나, 검색 엔진의 품직을 높이거나, 말뭉치에서 구조적인 데이터를 생성하는 데 활용할 수 있다.

5. 네 개의 공용어를 사용하여 스위스에서 주로 활동하는 고객을 위해 NER을 수행한다고 가정한다.